In [6]:
#!pip install pmdarima
#!pip install tqdm xgboost ta statsmodels numpy pandas matplotlib
from tqdm import tqdm
import random
import xgboost as xgb
from ta.momentum import RSIIndicator,StochasticOscillator
from statsmodels.tsa.arima.model import ARIMA

#Exercise1
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
# 设置初始参数
S0 = 100         # 初始股价
mu = 0.05        # 年化收益率
sigma = 0.2      # 年化波动率
days = 200     # 模拟天数
dt = 1/252       # 每天作为交易日的比例（通常一年252个交易日）

np.random.seed(0)  # 设置随机种子

# 生成每天的样本路径
price_paths = [S0]
for _ in range(1, days):
    z = np.random.standard_normal()  # 生成正态分布随机数
    S_t = price_paths[-1] * np.exp((mu - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z)
    price_paths.append(S_t)

# 根据生成的路径模拟开盘、收盘、最高、最低价格
stock_data = {'Open': [], 'Close': [], 'High': [], 'Low': []}
for i in tqdm(range(1, len(price_paths))):
    daily_prices = np.random.lognormal(mean=np.log(price_paths[i-1]), sigma=sigma*np.sqrt(dt), size=4)
    stock_data['Open'].append(daily_prices[0])
    stock_data['Close'].append(daily_prices[1])
    stock_data['High'].append(np.max(daily_prices))
    stock_data['Low'].append(np.min(daily_prices))

# 转换为DataFrame
stock_df = pd.DataFrame(stock_data)
start = "2023-01-01"
stock_df.index = pd.date_range(start=start, periods=days-1, freq='D')
print(stock_df.head())

# 绘制股价路径图
plt.figure(figsize=(10, 5))
plt.plot(stock_df['Close'], label='Close Price')
plt.fill_between(stock_df.index, stock_df['Low'], stock_df['High'], color='gray', alpha=0.3)
plt.title('Simulated Stock Price - Geometric Brownian Motion')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()


ImportError: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.8 from "/opt/miniconda3/envs/Morgan_Stanley/bin/python"
  * The NumPy version is: "1.24.4"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: dlopen(/opt/miniconda3/envs/Morgan_Stanley/lib/python3.8/site-packages/numpy/core/_multiarray_umath.cpython-38-darwin.so, 0x0002): Library not loaded: @rpath/libcblas.3.dylib
  Referenced from: <B73920F7-9293-3B5D-98B8-E470831CDC98> /opt/miniconda3/envs/Morgan_Stanley/lib/python3.8/site-packages/numpy/core/_multiarray_umat

In [7]:
ret = stock_df.pct_change().dropna()
ret

NameError: name 'stock_df' is not defined

In [ ]:
"""
1. 初始投资禀赋的不同体现在两个方面，一是信息获取能力的强弱，二是风险厌恶程度
第一类投资者信息获取能力强，可以通过机器学习，获取上市股票较为准确的信息分析挖掘多种因子作为输入量来预测
第二类投资者信息获取能力差，只能通过趋势学习，基本面分析来预测股票走势
第三类投资者随机应变投资者，对价格的预测根据上一日的收盘价加上一个正态分布噪音，完全随机。

1.1 假设每类投资者都有100人，初始财富都为100股的股票+10000元，股票的初始价值为100元每股，模拟100天结果。
在实际买卖中会有佣金 slippery cost等，在这里我们简化了

2. 申购行为：买卖报价的基本原则是当投资者作为买（卖）方进入市场时，若观测到的最近n次成交的价格低（高）于自己对当日收盘价的预测，则提交一个介于近期成交平均价与自己预测价之间的价格以提高买入的概率；若观测到的最近n次成交的价格高（低）于自己对当日收盘价的预测，则提交一个介于近期成交平均价与自己预测价之间的价格以提高卖出的概率。
bid：卖    ask：买

2.2 限价订单规则
具体地，进入市场的订单按价格优先和时间优先的准则在订单簿中排序，最优买入申报价格为最 高买入申报价格，记为Ｂ；最优卖出申报价格为最低卖出申报价格，记为 A。若新进入的买单报价>= A, 则以 A 的价格成交，即 q(现价）= A; 若新进入的卖单报价<=B,则以B的价格成交，即q(现价) = B。若最优买入申报价格与最优卖出申报价格相同，则q(现价）= A= B。
各类投资者以随机顺序进入市场，在所有投资者完成买卖选择后，一个交易日结束，清算每位投资着的总资产=现金+股票持有数量*股票当日收盘价格。随后进行下一个交易日的的模拟。

3. 分析结果： 收集和分析100天后每位投资者的最终财富状况，以及股票价格的变动趋势。

OPTION：1.每位投资者可以加入风险厌恶程度lambda，每次买入股票只会拿出总财富（w）的lambda * w来买交易。
"""

In [ ]:
# 生成初始的信息
def origin_info():
    #三类投资者数量 n_1,n_2,n_3
    np.random.seed(0)
    n_1 = 50
    n_2 = 50
    n_3 = 50
    n = n_1 + n_2 + n_3
    # 投资者分类列表
    list_1 = []
    # 每人资金列表
    list_2 = [float(10000) for _ in range(n)]
    # 每人股票数量列表
    list_3 = [float(100) for _ in range(n)]
    # 每人拥有的财富列表
    list_4 = [float(20000) for _ in range(n)]

    #风险厌恶系数，ratio越小表明对风险厌恶程度高
    weights = [round(np.random.uniform(0.5,1),2) for _ in range(n)]
    
    for i in range(n_1):
        list_1.append(1)
    for i in range(n_2):
        list_1.append(2)
    for i in range(n_3):
        list_1.append(3)

    dictionary = {'type': list_1, 'cash': list_2, 'holdings': list_3, 'wealth': list_4,"risk_tolerance": weights}
    investor_info = pd.DataFrame(dictionary)

    #初始化
    multi_index = pd.MultiIndex.from_product([stock_df.index, investor_info.index], names=['Date', 'Investor'])
    df_multi = pd.DataFrame(index=multi_index)
    # Populate the new DataFrame with data from investor_info and df2
    df_multi['Open'] = stock_df['Open'].reindex(df_multi.index, level='Date')
    df_multi['type'] = investor_info['type'].reindex(df_multi.index, level='Investor')
    df_multi['cash'] = investor_info['cash'].reindex(df_multi.index, level='Investor')
    df_multi['holdings'] = investor_info['holdings'].reindex(df_multi.index, level='Investor')
    df_multi['wealth'] = investor_info['wealth'].reindex(df_multi.index, level='Investor')
    df_multi['risk_tolerance'] = investor_info['risk_tolerance'].reindex(df_multi.index, level='Investor')
    df_multi["Prediction"] = float(0)
    return df_multi


In [ ]:
stock_df

In [ ]:
def generate_order(recent_prices, prediction):
    avg_recent_price = np.mean(recent_prices)

    if prediction > avg_recent_price:
        # 若预测价格高于近期平均成交价，投资者可能认为价格将上升，更倾向于买入
        difference = (prediction - avg_recent_price)
        temp = random.uniform(-difference, difference)
        bid_price = avg_recent_price+temp
        return ('buy', bid_price)
    else:
        # 若预测价格低于近期平均成交价，投资者可能认为价格将下降，更倾向于卖出
        difference = (avg_recent_price - prediction)
        temp = random.uniform(-difference, difference)
        ask_price = avg_recent_price+temp
        return ('sell', ask_price)

class OrderBook:
    def __init__(self):
        self.buy_orders = []
        self.sell_orders = []

    def add_order(self, order_type, price, investor):
        order = {'type': order_type, 'price': price, 'investor': investor}
        if order_type == 'buy':
            self.buy_orders.append(order)
        else:
            self.sell_orders.append(order)
            
    def sort(self):
        self.buy_orders.sort(key=lambda x: (-x['price'], x['investor']))  # Highest price first, earliest first if same price
        self.sell_orders.sort(key=lambda x: (x['price'], x['investor']))  # Lowest price first, earliest first if same price

    def match_orders(self):
        trades = []
        while self.buy_orders and self.sell_orders and self.buy_orders[0]['price'] >= self.sell_orders[0]['price']:
            buy_order = self.buy_orders.pop(0)
            sell_order = self.sell_orders.pop(0)
            trade_price = sell_order['price']  # Execute at the lowest ask price
            trades.append((buy_order['investor'], sell_order['investor'], trade_price))
            # Here you can implement trade execution logic and update accounts
        return trades


In [ ]:
def featurecalculator(share):
    share['EMA_9'] = share['Open'].ewm(9).mean() # exponential moving average of window 9
    share['SMA_5'] = share['Open'].rolling(5).mean() # moving average of window 5
    share['SMA_10'] = share['Open'].rolling(10).mean() # moving average of window 10
    share['SMA_15'] = share['Open'].rolling(15).mean() # moving average of window 15
    share['SMA_20'] = share['Open'].rolling(20).mean() # moving average of window 20
    share['SMA_25'] = share['Open'].rolling(25).mean() # moving average of window 25
    share['SMA_30'] = share['Open'].rolling(30).mean() # moving average of window 30
    EMA_12 = pd.Series(share['Open'].ewm(span=12, min_periods=12).mean())
    EMA_26 = pd.Series(share['Open'].ewm(span=26, min_periods=26).mean())
    share['MACD'] = pd.Series(EMA_12 - EMA_26)    # calculates Moving Average Convergence Divergence
    share['RSI'] = RSIIndicator(share['Open']).rsi() # calculates Relative Strength Index 
    share['Stochastic']=StochasticOscillator(share['High'],share['Low'],share['Open']).stoch() # Calculates Stochastic Oscillator
    pass

def ml_model(data):
    featurecalculator(data)
    data.dropna(inplace = True)
    data.drop(['High','Low','Close'],inplace=True,axis=1)
    # Preparing features and target
    data['Open'] = data['Open'].shift(-1)  # Predict next day's close
    training_data = data.iloc[:-1]
    X_train = training_data.drop(["Open"],axis=1)
    y_train = training_data["Open"]
    
    #use last day's feature for next day's price prediction
    features_for_prediction = data.drop(["Open"],axis = 1).iloc[-1].values.reshape(1, -1)
    
    # XGBoost model
    model = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree=0.4, learning_rate=0.1, max_depth=3, alpha=10, n_estimators=100)
    # Fit model
    model.fit(X_train, y_train)
    #predict price
    predicted_price = model.predict(features_for_prediction)
    return predicted_price

def trend_analysis(historical_data):
    model = ARIMA(historical_data, order=(1, 1, 1))
    model_fit = model.fit()
    forecasts = model_fit.forecast(steps=1)
    return forecasts.iloc[0]

def predict_prices(investor_type, historical):
    historical_data = historical.copy()
    if investor_type == 1:
        # 使用机器学习模型预测
        prediction = ml_model(historical_data)[0]
    elif investor_type == 2:
        historical_data = historical_data["Open"]
        # 使用基本面分析预测
        prediction = trend_analysis(historical_data)
    else:
        # 随机预测
        historical_data = historical_data["Open"]
        prediction = historical_data.iloc[-1] * np.random.normal(1, 0.05)  # 假设5%的波动
    return prediction

#Modifies df_multiple inplace
def update_accounts(df_multiple, trades, date):
    previous = date - timedelta(days = 1)
    columns_to_copy = ['cash','holdings', 'wealth']
    
    # Initialize today's financial state with yesterday's data if not already present
    df_multiple.loc[(date, slice(None)), columns_to_copy] = df_multiple.loc[(previous, slice(None)), columns_to_copy].values
        
    for buyer, seller, trade_price in trades:
        # Determine the maximum number of stocks that can be traded
        # Assuming each trade involves one stock unit by default unless specified otherwise
        
        buyer_cash_available = df_multiple.loc[(date, buyer), 'cash']
        seller_stocks_available = df_multiple.loc[(date, seller), 'holdings']

        # Calculate the number of stocks that can be bought by the buyer
        stocks_to_trade = min(buyer_cash_available // trade_price, seller_stocks_available)

        # Update buyer's details in df_multiple
        df_multiple.loc[(date, buyer), 'cash'] -= trade_price * stocks_to_trade
        df_multiple.loc[(date, buyer), 'holdings'] += stocks_to_trade
        df_multiple.loc[(date, buyer), 'wealth'] = df_multiple.loc[(date, buyer), 'cash'] + df_multiple.loc[(date, buyer), 'holdings'] * trade_price

        # Update seller's details in df_multi
        df_multiple.loc[(date, seller), 'cash'] += trade_price * stocks_to_trade
        df_multiple.loc[(date, seller), 'holdings'] -=  stocks_to_trade
        df_multiple.loc[(date, seller), 'wealth'] = df_multiple.loc[(date, seller), 'cash'] + df_multiple.loc[(date, seller), 'holdings'] * trade_price
    pass

In [ ]:
def strategy(df_multiple, test_period) -> int:
    stock_df_copy = stock_df.copy()
    Open_price = stock_df["Open"]
    all_dates = stock_df.index
    test_date = all_dates[-test_period:]
    investor_num = df_multiple.index.get_level_values(1).unique().values
    np.random.shuffle(investor_num)
    
    #每天为一个交易日，模拟100天的交易
    for i in tqdm(range(len(test_date))):
        date = test_date[i]
        np.random.shuffle(investor_num)
        order_book = OrderBook()
        
        for investor in investor_num:

            #为每一位投资者预测下一天的价格根据historical price
            investor_type = df_multiple.xs(investor,level = 1).iloc[0,1]
            prediction = predict_prices(investor_type, stock_df_copy.loc[start:date])
            df_multiple.loc[(date,investor),"Prediction"]= prediction

            # 获取最近n次的成交价格
            n = 3
            recent_prices = Open_price.loc[date-timedelta(days = n-1):date].values
            
            #generate order
            order_type, price = generate_order(recent_prices, prediction)
            order_book.add_order(order_type,price,investor)
        
        order_book.sort()
        trades = order_book.match_orders()
        
        update_accounts(df_multiple,trades,date)
    pass
    
def main():
    df_multiple = origin_info()
    #最后五十天为交易周期
    test_period = 100
    strategy(df_multiple,test_period)
    test_period = stock_df.index[-(test_period+1):]
    return df_multiple.loc[test_period]
df = main()

In [ ]:
df

In [ ]:
# Plot setup
plt.figure(figsize=(10,6))

# Loop through each investor and plot their wealth data
for investor_no in range(50):
    wealth = df.xs(investor_no, level=1)['wealth']
    plt.plot(wealth.index, wealth, label=f'Investor {investor_no}')

# Optionally, you can uncomment the following line to display a legend
# plt.legend()

plt.title("Wealth Change for Investors 1-100")
plt.xlabel("Date")
plt.ylabel("Wealth")
plt.legend()
plt.show()

In [ ]:
# coding=utf-8
import jqdatasdk as jq
import pandas as pd
import numpy as np
import cvxpy as cp
from datetime import datetime
from scipy import stats, integrate

# Authentication with JoinQuant
jq.auth('18084028253', 'Ch214199')

# Function to fetch data using JoinQuant
def fetch_data(code, start_date='20230116', end_date='20240123'):
    df = jq.get_price(code, start_date=start_date, end_date=end_date, frequency='daily', fields=['close'])
    df['close'] = df['close'] / df['close'].iloc[0]  # Normalize to calculate net value
    return df['close']

# Fetch data for different indices
#上证商品
ret_nh = fetch_data("000066.XSHG").values  # Adjust the symbol according to actual availability in JoinQuant
#沪深300
ret_hs300 = fetch_data('000300.XSHG').values
#上证国债指数000012
ret_sz = fetch_data('000012.XSHG').values  # Shenzhen Component Index

# df=jq.get_all_securities(types=['index'])
# df[df.display_name.str.contains("商品")]

In [ ]:
price = np.stack([ret_hs300, ret_sz, ret_nh], axis=1)
price = pd.DataFrame(data = price, columns=["沪深300","上证国债指数","上证商品期货"])

ret = price.pct_change().dropna()
ret

In [14]:
#Task 2&3.对选取的三只asset进行收益率数据分析，进行给定预期收益的最佳持仓配比求解
from scipy import optimize

# Find optimal portfolio with lowest variance return:
# 修改权重和为1的约束函数，使其不接受外部参数
def weights_const(w):
    return np.sum(w) - 1

# 返回满足特定收益率R的约束函数
# def return_constraint(R):
#     def constraint(w):
#         return np.sum(ret.mean() * w) * 252 - R
#     return constraint

# 方差最小化函数不变
def get_vola(w):
    w = np.array(w)
    vola_w = np.sqrt(np.dot(w.T, np.dot(ret.cov() * 252, w)))
    return vola_w

def get_ret(w):
    w = np.array(w)
    ret_w = np.sum(ret.mean() * w) * 252
    return ret_w

def optimize_portfolio(bounds, x0):
    constraints = (
        {'type': 'eq', 'fun': weights_const}  # 使用R作为参数
    )
    optim_ptf = optimize.minimize(get_vola, x0, method='SLSQP', bounds=bounds, constraints=constraints)
    return optim_ptf

const = ({'type': 'eq', 'fun': weights_const}) # optimization constraints
bounds = ((0, 1), (0, 1), (0, 1)) # weights bounds
x0 = [0.33, 0.33, 0.33] # starting point for optimization
optim_ptf = optimize_portfolio(bounds,x0)
opt_ret = np.sum(ret.mean() * optim_ptf.x) * 252
opt_vola = np.sqrt(np.dot(optim_ptf.x.T, np.dot(ret.cov() * 252, optim_ptf.x)))
opt_sr = opt_ret / opt_vola

print("Optimal weights:", np.round(optim_ptf.x*100,2), "%")
print("Optimal return:", opt_ret)
print("Optimal volatility:", opt_vola)
print("Optimal Sharpe ratio:", opt_sr)
print("股票： %.2f  债券： %.2f    商品期货：%.2f" %(optim_ptf.x[0],optim_ptf.x[1],optim_ptf.x[2]))

NameError: name 'ret' is not defined

In [10]:
import sys
import os
import re
import glob
import datetime as dt

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

from mosek.fusion import *

#from notebook.services.config import ConfigManager

#from portfolio_tools import data_download, DataReader

In [16]:
# Define function solving the optimization model
def Markowitz(N, m, G, gamma2):
    with Model("markowitz") as M:
        # Settings
        M.setLogHandler(sys.stdout)

        # Decision variable (fraction of holdings in each security)
        # The variable x is restricted to be positive, which imposes the constraint of no short-selling.   
        x = M.variable("x", N, Domain.greaterThan(0.0))

        # Budget constraint
        M.constraint('budget', Expr.sum(x), Domain.equalsTo(1))

        # Objective 
        M.objective('obj', ObjectiveSense.Maximize, Expr.dot(m, x))

        # Imposes a bound on the risk
        #M.constraint('risk', Expr.vstack(gamma2, 0.5, Expr.mul(G.transpose(), x)), Domain.inRotatedQCone())

        # Solve optimization
        M.solve()

        # Check if the solution is an optimal point
        solsta = M.getPrimalSolutionStatus()
        if (solsta != SolutionStatus.Optimal):
            # See https://docs.mosek.com/latest/pythonfusion/accessing-solution.html about handling solution statuses.
            raise Exception("Unexpected solution status!")

        returns = M.primalObjValue()
        portfolio = x.level()

    return returns, portfolio

In [17]:
mu = np.array([0.07197349, 0.15518171, 0.17535435, 0.0898094 , 0.42895777, 0.39291844, 0.32170722, 0.18378628])
Sigma = np.array([
    [0.09460323, 0.03735969, 0.03488376, 0.03483838, 0.05420885, 0.03682539, 0.03209623, 0.03271886],
    [0.03735969, 0.07746293, 0.03868215, 0.03670678, 0.03816653, 0.03634422, 0.0356449 , 0.03422235],
    [0.03488376, 0.03868215, 0.06241065, 0.03364444, 0.03949475, 0.03690811, 0.03383847, 0.02433733],
    [0.03483838, 0.03670678, 0.03364444, 0.06824955, 0.04017978, 0.03348263, 0.04360484, 0.03713009],
    [0.05420885, 0.03816653, 0.03949475, 0.04017978, 0.17243352, 0.07886889, 0.06999607, 0.05010711],
    [0.03682539, 0.03634422, 0.03690811, 0.03348263, 0.07886889, 0.09093307, 0.05364518, 0.04489357],
    [0.03209623, 0.0356449 , 0.03383847, 0.04360484, 0.06999607, 0.05364518, 0.09649728, 0.04419974],
    [0.03271886, 0.03422235, 0.02433733, 0.03713009, 0.05010711, 0.04489357, 0.04419974, 0.08159633]
])

N = mu.shape[0]  # Number of securities
gamma2 = 0.05   # Risk limit (variance)

G = np.linalg.cholesky(Sigma)  # Cholesky factor of S to use in conic risk constraint 
# Run optimization 
f, x = Markowitz(N, mu, G, gamma2)
print("========================\n")
print("RESULTS:")
print(f"Optimal expected portfolio return: {f*100:.4f}%")
print(f"Optimal portfolio weights: {x}")
print(f"Sum of weights: {np.sum(x)}")

Problem
  Name                   : markowitz       
  Objective sense        : maximize        
  Type                   : LO (linear optimization problem)
  Constraints            : 1               
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 9               
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 0                 time                   : 0.00            
Lin. dep.  - primal attempts        : 0                 successes              : 0               
Lin. dep.  - dual attempts          : 0                 successes              : 0               
Lin. dep.  - primal deps.  

In [15]:
temp,temp1 = f,x
print(f"Optimal expected portfolio return: {temp*100:.4f}%")
print(f"Optimal portfolio weights: {temp1}")
print(f"Sum of weights: {np.sum(temp1)}")

Optimal expected portfolio return: 27.6717%
Optimal portfolio weights: [8.01278769e-08 9.12691841e-02 2.69097739e-01 1.14489110e-07
 2.52926398e-02 3.21630359e-01 1.76525071e-01 1.16184804e-01]
Sum of weights: 0.9999999915854186
